### A) Descarga datos:

```
Crear script en Python para descargar de forma automática datos de 2024 de https://www.prt.cl/Descargas/ , consolidar datos y presentar graficas con los datos para mostrar las marcas y modelos más vendidos.
```



In [1]:
import os
import requests
import zipfile
import pandas as pd 

A1 = "SGPRT_RA1"
A2 = "SGPRT_RA2"
B = "SGPRT_RB"

years = [2024]
months = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11:"Noviembre",
    12: "Diciembre"
}

In [2]:
# To obtain the URL given the year and the month number
# It returns a dict: {filenameA1: urlA1, filenameA2: urlA2, filenameB: urlB}

def generate_urls(year, month_num):
    month_name = months.get(month_num)
    month_abbreviation = month_name.lower()[:3]

    # Formatting
    file_names = {
        f"{A1}_{month_abbreviation}-{year}": f"https://www.prt.cl/Descargas/docs/{year}/{month_num}.{month_name}/{A1}_{month_abbreviation}-{year}.zip",
        f"{A2}_{month_abbreviation}-{year}": f"https://www.prt.cl/Descargas/docs/{year}/{month_num}.{month_name}/{A2}_{month_abbreviation}-{year}.zip",
        f"{B}_{month_abbreviation}-{year}": f"https://www.prt.cl/Descargas/docs/{year}/{month_num}.{month_name}/{B}_{month_abbreviation}-{year}.zip"
    }

    return file_names

#### Download a single zip

- It download the file, uncompress the zip (on memory using zipfile), open the excel file and load the data into a Dataframe
- If can't download, then returns None

It uses ```engine="calamine"``` in ```pd.read_excel()``` provided by pandas_monkeypatch from ```python_calamine.pandas``` (sooo much faster)

In [7]:
from python_calamine.pandas import pandas_monkeypatch
pandas_monkeypatch()

def download_zip(file_name, url, download_dir):
    file_path = os.path.join(download_dir, f"{file_name}.zip")
    
    try:
        print(f"\tDownloading {file_name} from {url}...")
        response = requests.get(url)
        
        # Save file only if download is OK (200)
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(response.content)
            print(f"\t\tFile {file_name} was downloaded in {file_path}.")

            # If file was saved correctly, uncompress and load the excel file
            file = zipfile.ZipFile(f'data\{file_name}.zip', 'r')
            xlfile = file.open(f'{file_name}.xlsx')

            # Create a new Dataframe
            df_excel = pd.read_excel(xlfile, engine="calamine")
            print(f"\t\tThe file ({file_name}) was uncompressed and loaded correctly")
            return df_excel
        else:
            print(f"\t\tThe file {file_name} wasn't downloaded. \n\t\tStatus code: {response.status_code}")
            return None
    except Exception as e:
        print(f"\t\tThere was an error trying to download/open/load the file {file_name}. \n\tError: {e}")
        return None

#### Get data from dict with URLs (those 3 files to download per month)

If it is ok, then put the data into the dict ```data_dict[A1|A2|B]``` and return it

In [8]:
# file_urls is a dict: {filenameA1: url1, filenameA2: url2,... etc}

def get_data(file_urls, download_folder='data'):
    download_dir = os.path.join(os.getcwd(), download_folder)
    data_dict = {}

    # Check if folder exists, if not create it
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

    for file_name, url in file_urls.items():
        df_excel = download_zip(file_name, url, download_dir) # Could be None

        if A1 in file_name:
            data_dict[A1] = df_excel
        elif A2 in file_name:
            data_dict[A2] = df_excel
        elif B in file_name:
            data_dict[B] = df_excel
        else:
            print(f"\tThe file ({file_name}) doesn't match with existing names (A1, A2, B)")
            continue

    return data_dict

### Run run runnn (~10 min?)

- For every month ```{months}``` of every year ```{years}```, get data, download the zips and create a dataframe (if apply)
- Every time, concat the resulting dataframe to a existing one depending on the case

In [86]:
df_A1 = pd.DataFrame()
df_A2 = pd.DataFrame()
df_B = pd.DataFrame()

for year in years:
    print(f"\n{year=}")
    for month_num in months:
        print(f"\n{month_num=}")
        data_dict = get_data(generate_urls(year, month_num))

        for file_name, df_excel in data_dict.items():
            if df_excel is not None:
                # Concatenate with existing dataframes, depending on the case
                if A1 in file_name:
                    df_A1 = pd.concat([df_A1, df_excel], ignore_index=True)
                elif A2 in file_name:
                    df_A2 = pd.concat([df_A2, df_excel], ignore_index=True)
                else:
                    df_B = pd.concat([df_B, df_excel], ignore_index=True)


year=2024

month_num=1
		File SGPRT_RA1_ene-2024 was downloaded in d:\Programming\pruebastecnicas\AutoLab\a_descargar_datos\data\SGPRT_RA1_ene-2024.zip.
		The file (SGPRT_RA1_ene-2024) was uncompressed and loaded correctly
		File SGPRT_RA2_ene-2024 was downloaded in d:\Programming\pruebastecnicas\AutoLab\a_descargar_datos\data\SGPRT_RA2_ene-2024.zip.
		The file (SGPRT_RA2_ene-2024) was uncompressed and loaded correctly
		File SGPRT_RB_ene-2024 was downloaded in d:\Programming\pruebastecnicas\AutoLab\a_descargar_datos\data\SGPRT_RB_ene-2024.zip.
		The file (SGPRT_RB_ene-2024) was uncompressed and loaded correctly

month_num=2
		File SGPRT_RA1_feb-2024 was downloaded in d:\Programming\pruebastecnicas\AutoLab\a_descargar_datos\data\SGPRT_RA1_feb-2024.zip.
		The file (SGPRT_RA1_feb-2024) was uncompressed and loaded correctly
		File SGPRT_RA2_feb-2024 was downloaded in d:\Programming\pruebastecnicas\AutoLab\a_descargar_datos\data\SGPRT_RA2_feb-2024.zip.
		The file (SGPRT_RA2_feb-2024) was 

## Finally
- Make fancy graphs using ChatGPT¿¿
- Profit

In [87]:
df_A1['MARCA'].value_counts()

MARCA
MERCEDES BENZ         50889
SCANIA                 6133
VOLVO                  5628
FORD                   4421
VOLARE                 4230
                      ...  
YONGMAN                   1
VOLKSWAGWEN               1
PULLMAN STD AMERIC        1
STANDARD                  1
MILLER                    1
Name: count, Length: 153, dtype: int64

In [89]:
top_marca = df_A1['MARCA'].value_counts().head().index
top_marca

Index(['MERCEDES BENZ', 'SCANIA', 'VOLVO', 'FORD', 'VOLARE'], dtype='object', name='MARCA')

In [88]:
df_A1[['MARCA', 'MODELO']][df_A1['MARCA'].isin(top_marca)]

,MARCA,MODELO
2,MERCEDES BENZ,LO 915
3,MERCEDES BENZ,OF 1620
4,MERCEDES BENZ,LO 916
5,MERCEDES BENZ,LO 915
6,MERCEDES BENZ,LO 916 45
...,...,...
98320,FORD,TRANSIT
98321,MERCEDES BENZ,SPRINTER
98322,FORD,TRANSIT 2.2 L
98323,MERCEDES BENZ,SPRINTER


In [17]:
df_A2.columns

Index(['COD_PRT', 'PPU', 'COD_VEHICULO', 'COD_COMBUSTIBLE', 'COD_SERVICIO',
       'MARCA', 'MODELO', 'ANO_FABRICACION', 'NUM_MOTOR', 'NUM_CHASIS', 'VIN',
       'KILOMETRAJE', 'NUM_CERTIFICADO', 'FEC_REVISION', 'FEC_VENCIMIENTO',
       'HORA_INI', 'HORA_FIN', 'RESULTADO_CRT', 'FEC_VENCIMIENTO_GASES',
       'RESULTADO_CRT_GASES', 'IDENTIFICACION', 'VISUAL', 'LUCES',
       'ALINEACION', 'FRENOS', 'HOLGURAS', 'SUSPENSION', 'GASES', 'OPACIDAD',
       'ANGULO_GIRO', 'V_CO_RALENTI', 'R_CO_RALENTI', 'V_HC_RALENTI',
       'R_HC_RALENTI', 'V_COCO2_RALENTI', 'R_COCO2_RALENTI', 'V_CO_2500RPM',
       'R_CO_2500RPM', 'V_HC_2500RPM', 'R_HC_2500RPM', 'V_COCO2_2500RPM',
       'R_COCO2_2500RPM', 'V_HC_STDR5015', 'R_HC_STDR5015', 'V_CO_STDR5015',
       'R_CO_STDR5015', 'V_NO_STDR5015', 'R_NO_STDR5015', 'V_HC_STDR2525',
       'R_HC_STDR2525', 'V_CO_STDR2525', 'R_CO_STDR2525', 'V_NO_STDR2525',
       'R_NO_STDR2525', 'R_HUMO', 'V_OPA1', 'V_OPA2', 'V_OPA3', 'V_OPA4',
       'V_OPA5', 'V_VALIDA_OP